# Upload / Download Images

In [7]:
import boto3
import botocore
from os import listdir

PROFILE = "innovacion-dev-developer"
BUCKET = "icemd-awsintro"
FOLDER = "s3" 
REGION = "eu-west-1"
WITH_PROFILE = 0
ACTION = "DOWNLOAD" # "UPLOAD" "DOWNLOAD"

In [8]:
if WITH_PROFILE:
    session = boto3.Session(profile_name=PROFILE)
    s3 = session.client("s3")
else:
    s3 = boto3.client("s3")

In [9]:
buckets_list = s3.list_buckets()['Buckets']
buckets_names = [buckets_list[i]['Name'] for i in range(0,len(buckets_list))]

# Create S3 bucket called BUCKET
if BUCKET not in buckets_names:
    s3.create_bucket(Bucket=BUCKET, CreateBucketConfiguration={'LocationConstraint': REGION})

NoCredentialsError: Unable to locate credentials

In [15]:
if ACTION == "UPLOAD":
    # Upload files to S3_FOLDER in BUCKET
    files = listdir(FOLDER, )
    files_names = [file for file in files if file.endswith('.jpg') or file.endswith('.mp4')]
    for name in files_names:
        s3.upload_file(FOLDER + name, BUCKET, FOLDER + name)      
else if ACTION == "DOWNLOAD":
    # Download files to FOLDER from S3_FOLDER in BUCKET
    objects_list = s3.list_objects(Bucket = BUCKET)
    objects_keys = [key['Key'].encode("UTF-8") for key in objects_list['Contents']]
    for image in objects_keys:
        try:
            s3.download_file(BUCKET, image, FOLDER + image)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist")
            else:
                raise
else:
    print("Invalid option")